<a href="https://colab.research.google.com/github/ryudonggle/lg-aimers-5-/blob/main/Aimer_phase2_final_sub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 최종


In [ ]:
!pip install --upgrade pip

!pip install numpy==1.26.4
!pip install pandas==2.1.4
!pip install scikit-learn==1.3.2
!pip install catboost==1.2.5
!pip install optuna==3.6.1
!pip install xgboost==2.1.1
!pip install lightgbm==4.4.0

In [ ]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

기존 데이터 EDA 과정에서 특정 열 이후로 시프트 된 것으로 추정, 수정한 데이터 사용

In [ ]:
ROOT_DIR = "data"
RANDOM_STATE = 110

# 데이터 읽어오기
df = pd.read_csv(os.path.join(ROOT_DIR, "new_train.csv"))
test_df = pd.read_csv(os.path.join(ROOT_DIR, "new_test.csv"))

In [ ]:
#결측치 제거 - 모든 행이 결측치면 삭제
dr=df.isnull().sum().sort_values()[-281:].index
df.drop(columns=dr,inplace=True)
test_df.drop(columns=dr,inplace=True)

#고유값이 하나면 모델 학습에 영향을 주지 못할 것이라 판단, 삭제
dr_0=df.nunique().sort_values()[:64].index
df.drop(columns=dr_0,inplace=True)
test_df.drop(columns=dr_0,inplace=True)

In [ ]:
# 파생 변수 생성

# 합계/평균/최대/최소/표준편차/분산/비율

#Dam
df['MEAN DISCHARGED TIME OF RESIN_Dam']=df[['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']].mean(axis=1)
df['STD DISCHARGED TIME OF RESIN_Dam']=df[['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']].std(axis=1)
df['SUM DISCHARGED TIME OF RESIN_Dam']=df[['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']].sum(axis=1)

df['MEAN Dispense Volume_Dam']=df[['Dispense Volume(Stage1) Collect Result_Dam',
       'Dispense Volume(Stage2) Collect Result_Dam',
       'Dispense Volume(Stage3) Collect Result_Dam']].mean(axis=1)
df['STD Dispense Volume_Dam']=df[['Dispense Volume(Stage1) Collect Result_Dam',
       'Dispense Volume(Stage2) Collect Result_Dam',
       'Dispense Volume(Stage3) Collect Result_Dam']].std(axis=1)
df['SUM Dispense Volume_Dam']=df[['Dispense Volume(Stage1) Collect Result_Dam',
       'Dispense Volume(Stage2) Collect Result_Dam',
       'Dispense Volume(Stage3) Collect Result_Dam']].sum(axis=1)

#비율 - 같은 용량이 나오는 데에 걸리는 시간
df['DIV Dispense Volume(Stage1)_Dam']=df['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam']/df['Dispense Volume(Stage1) Collect Result_Dam']
df['DIV Dispense Volume(Stage2)_Dam']=df['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam']/df['Dispense Volume(Stage2) Collect Result_Dam']
df['DIV Dispense Volume(Stage3)_Dam']=df['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']/df['Dispense Volume(Stage3) Collect Result_Dam']

#Fill1
#합계/평균/최대/최소/표준편차/분산/비율

df['MEAN DISCHARGED TIME OF RESIN_Fill1']=df[['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1',
       'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1',
       'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']].mean(axis=1)
df['STD DISCHARGED TIME OF RESIN_Fill1']=df[['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1',
       'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1',
       'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']].std(axis=1)
df['SUM DISCHARGED TIME OF RESIN_Fill1']=df[['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1',
       'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1',
       'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']].sum(axis=1)

df['MEAN Dispense Volume_Fill1']=df[['Dispense Volume(Stage1) Collect Result_Fill1',
       'Dispense Volume(Stage2) Collect Result_Fill1',
       'Dispense Volume(Stage3) Collect Result_Fill1']].mean(axis=1)
df['STD Dispense Volume_Fill1']=df[['Dispense Volume(Stage1) Collect Result_Fill1',
       'Dispense Volume(Stage2) Collect Result_Fill1',
       'Dispense Volume(Stage3) Collect Result_Fill1']].std(axis=1)
df['SUM Dispense Volume_Fill1']=df[['Dispense Volume(Stage1) Collect Result_Fill1',
       'Dispense Volume(Stage2) Collect Result_Fill1',
       'Dispense Volume(Stage3) Collect Result_Fill1']].sum(axis=1)

#비율 - 같은 용량이 나오는 데에 걸리는 시간
df['DIV Dispense Volume(Stage1)_Fill1']=df['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1']/df['Dispense Volume(Stage1) Collect Result_Fill1']
df['DIV Dispense Volume(Stage2)_Fill1']=df['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1']/df['Dispense Volume(Stage2) Collect Result_Fill1']
df['DIV Dispense Volume(Stage3)_Fill1']=df['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']/df['Dispense Volume(Stage3) Collect Result_Fill1']

#Thickness(변화가 있는지도 비율로 보면 좋을듯)

df['SUM THICKNESS_Dam']=df[['THICKNESS 1 Collect Result_Dam', 'THICKNESS 2 Collect Result_Dam','THICKNESS 3 Collect Result_Dam']].sum(axis=1)
df['MEAN THICKNESS_Dam']=df[['THICKNESS 1 Collect Result_Dam', 'THICKNESS 2 Collect Result_Dam','THICKNESS 3 Collect Result_Dam']].mean(axis=1)
df['STD THICKNESS_Dam']=df[['THICKNESS 1 Collect Result_Dam', 'THICKNESS 2 Collect Result_Dam','THICKNESS 3 Collect Result_Dam']].std(axis=1)

df['DIV1 THICKNESS_Dam']=df['THICKNESS 2 Collect Result_Dam']/df['THICKNESS 1 Collect Result_Dam']
df['DIV2 THICKNESS_Dam']=df['THICKNESS 3 Collect Result_Dam']/df['THICKNESS 2 Collect Result_Dam']

#1st/2nd/3rd - 'THICKNESS 1 Collect Result_Dam', 'THICKNESS 2 Collect Result_Dam''THICKNESS 3 Collect Result_Dam'
df['SUM Pressure_AutoClave']=df[['1st Pressure Collect Result_AutoClave', '2nd Pressure Collect Result_AutoClave', '3rd Pressure Collect Result_AutoClave']].sum(axis=1)
df['MEAN Pressure_AutoClave']=df[['1st Pressure Collect Result_AutoClave', '2nd Pressure Collect Result_AutoClave', '3rd Pressure Collect Result_AutoClave']].mean(axis=1)
df['STD Pressure_AutoClave']=df[['1st Pressure Collect Result_AutoClave', '2nd Pressure Collect Result_AutoClave', '3rd Pressure Collect Result_AutoClave']].std(axis=1)

df['DIV1 Pressure_AutoClave']=df['2nd Pressure Collect Result_AutoClave']/df['1st Pressure Collect Result_AutoClave']
df['DIV2 Pressure_AutoClave']=df['3rd Pressure Collect Result_AutoClave']/df['2nd Pressure Collect Result_AutoClave']

In [ ]:
#합계/평균/최대/최소/표준편차/분산/비율
#Dam
test_df['MEAN DISCHARGED TIME OF RESIN_Dam']=test_df[['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']].mean(axis=1)
test_df['STD DISCHARGED TIME OF RESIN_Dam']=test_df[['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']].std(axis=1)
test_df['SUM DISCHARGED TIME OF RESIN_Dam']=test_df[['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']].sum(axis=1)

test_df['MEAN Dispense Volume_Dam']=test_df[['Dispense Volume(Stage1) Collect Result_Dam',
       'Dispense Volume(Stage2) Collect Result_Dam',
       'Dispense Volume(Stage3) Collect Result_Dam']].mean(axis=1)
test_df['STD Dispense Volume_Dam']=test_df[['Dispense Volume(Stage1) Collect Result_Dam',
       'Dispense Volume(Stage2) Collect Result_Dam',
       'Dispense Volume(Stage3) Collect Result_Dam']].std(axis=1)
test_df['SUM Dispense Volume_Dam']=test_df[['Dispense Volume(Stage1) Collect Result_Dam',
       'Dispense Volume(Stage2) Collect Result_Dam',
       'Dispense Volume(Stage3) Collect Result_Dam']].sum(axis=1)

#비율 - 같은 용량이 나오는 데에 걸리는 시간
test_df['DIV Dispense Volume(Stage1)_Dam']=test_df['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam']/test_df['Dispense Volume(Stage1) Collect Result_Dam']
test_df['DIV Dispense Volume(Stage2)_Dam']=test_df['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam']/test_df['Dispense Volume(Stage2) Collect Result_Dam']
test_df['DIV Dispense Volume(Stage3)_Dam']=test_df['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam']/test_df['Dispense Volume(Stage3) Collect Result_Dam']

#Fill1
#합계/평균/최대/최소/표준편차/분산/비율

test_df['MEAN DISCHARGED TIME OF RESIN_Fill1']=test_df[['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1',
       'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1',
       'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']].mean(axis=1)
test_df['STD DISCHARGED TIME OF RESIN_Fill1']=test_df[['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1',
       'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1',
       'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']].std(axis=1)
test_df['SUM DISCHARGED TIME OF RESIN_Fill1']=test_df[['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1',
       'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1',
       'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']].sum(axis=1)

test_df['MEAN Dispense Volume_Fill1']=test_df[['Dispense Volume(Stage1) Collect Result_Fill1',
       'Dispense Volume(Stage2) Collect Result_Fill1',
       'Dispense Volume(Stage3) Collect Result_Fill1']].mean(axis=1)
test_df['STD Dispense Volume_Fill1']=test_df[['Dispense Volume(Stage1) Collect Result_Fill1',
       'Dispense Volume(Stage2) Collect Result_Fill1',
       'Dispense Volume(Stage3) Collect Result_Fill1']].std(axis=1)
test_df['SUM Dispense Volume_Fill1']=test_df[['Dispense Volume(Stage1) Collect Result_Fill1',
       'Dispense Volume(Stage2) Collect Result_Fill1',
       'Dispense Volume(Stage3) Collect Result_Fill1']].sum(axis=1)

#비율 - 같은 용량이 나오는 데에 걸리는 시간
test_df['DIV Dispense Volume(Stage1)_Fill1']=test_df['DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1']/test_df['Dispense Volume(Stage1) Collect Result_Fill1']
test_df['DIV Dispense Volume(Stage2)_Fill1']=test_df['DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1']/test_df['Dispense Volume(Stage2) Collect Result_Fill1']
test_df['DIV Dispense Volume(Stage3)_Fill1']=test_df['DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']/test_df['Dispense Volume(Stage3) Collect Result_Fill1']

#Thickness(변화가 있는지도 비율로 보면 좋을듯)

test_df['SUM THICKNESS_Dam']=test_df[['THICKNESS 1 Collect Result_Dam', 'THICKNESS 2 Collect Result_Dam','THICKNESS 3 Collect Result_Dam']].sum(axis=1)
test_df['MEAN THICKNESS_Dam']=test_df[['THICKNESS 1 Collect Result_Dam', 'THICKNESS 2 Collect Result_Dam','THICKNESS 3 Collect Result_Dam']].mean(axis=1)
test_df['STD THICKNESS_Dam']=test_df[['THICKNESS 1 Collect Result_Dam', 'THICKNESS 2 Collect Result_Dam','THICKNESS 3 Collect Result_Dam']].std(axis=1)

test_df['DIV1 THICKNESS_Dam']=test_df['THICKNESS 2 Collect Result_Dam']/test_df['THICKNESS 1 Collect Result_Dam']
test_df['DIV2 THICKNESS_Dam']=test_df['THICKNESS 3 Collect Result_Dam']/test_df['THICKNESS 2 Collect Result_Dam']

#1st/2nd/3rd - 'THICKNESS 1 Collect Result_Dam', 'THICKNESS 2 Collect Result_Dam''THICKNESS 3 Collect Result_Dam'
test_df['SUM Pressure_AutoClave']=test_df[['1st Pressure Collect Result_AutoClave', '2nd Pressure Collect Result_AutoClave', '3rd Pressure Collect Result_AutoClave']].sum(axis=1)
test_df['MEAN Pressure_AutoClave']=test_df[['1st Pressure Collect Result_AutoClave', '2nd Pressure Collect Result_AutoClave', '3rd Pressure Collect Result_AutoClave']].mean(axis=1)
test_df['STD Pressure_AutoClave']=test_df[['1st Pressure Collect Result_AutoClave', '2nd Pressure Collect Result_AutoClave', '3rd Pressure Collect Result_AutoClave']].std(axis=1)

test_df['DIV1 Pressure_AutoClave']=test_df['2nd Pressure Collect Result_AutoClave']/test_df['1st Pressure Collect Result_AutoClave']
test_df['DIV2 Pressure_AutoClave']=test_df['3rd Pressure Collect Result_AutoClave']/test_df['2nd Pressure Collect Result_AutoClave']

In [ ]:
# 0으로 나누어 생기 nan값 0으로 처리
df['DIV1 THICKNESS_Dam']=df['DIV1 THICKNESS_Dam'].fillna(0)
df['DIV2 THICKNESS_Dam']=df['DIV2 THICKNESS_Dam'].fillna(0)

test_df['DIV1 THICKNESS_Dam']=test_df['DIV1 THICKNESS_Dam'].fillna(0)
test_df['DIV2 THICKNESS_Dam']=test_df['DIV2 THICKNESS_Dam'].fillna(0)
print(df['DIV1 THICKNESS_Dam'].isna().sum()) #확인

In [ ]:
# 사용할 컬럼

cols = [

        'Equipment_Dam', 'Model.Suffix_Dam', 'Workorder_Dam',
       'CURE END POSITION X Collect Result_Dam',
       'CURE END POSITION Z Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam',
       'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam',
       'Dispense Volume(Stage1) Collect Result_Dam',
       'Dispense Volume(Stage2) Collect Result_Dam',
       'Dispense Volume(Stage3) Collect Result_Dam',
       'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam',
       'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam',
       'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam',
       'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam',
       'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam',
       'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam',
       'HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam',
       'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam',
       'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam',
       'Machine Tact time Collect Result_Dam',
       'Production Qty Collect Result_Dam', 'Receip No Collect Result_Dam',
       'Stage1 Circle1 Distance Speed Collect Result_Dam',
       'Stage1 Line1 Distance Speed Collect Result_Dam',
       'Stage2 Circle1 Distance Speed Collect Result_Dam',
       'Stage2 Line1 Distance Speed Collect Result_Dam',
       'Stage3 Circle1 Distance Speed Collect Result_Dam',
       'Stage3 Line1 Distance Speed Collect Result_Dam',
       'Stage3 Line3 Distance Speed Collect Result_Dam',
       'THICKNESS 1 Collect Result_Dam', 'THICKNESS 2 Collect Result_Dam',
       'THICKNESS 3 Collect Result_Dam',
       '1st Pressure Collect Result_AutoClave',
       '1st Pressure 1st Pressure Unit Time_AutoClave',
       '2nd Pressure Collect Result_AutoClave',
       '2nd Pressure Unit Time_AutoClave',
       '3rd Pressure Collect Result_AutoClave',
       '3rd Pressure Unit Time_AutoClave',
       'Chamber Temp. Collect Result_AutoClave',
       'Chamber Temp. Unit Time_AutoClave',
       'Chamber Temp. Judge Value_AutoClave',
       'DISCHARGED SPEED OF RESIN Collect Result_Fill1',
       'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1',
       'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1',
       'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1',
       'Dispense Volume(Stage1) Collect Result_Fill1',
       'Dispense Volume(Stage2) Collect Result_Fill1',
       'Dispense Volume(Stage3) Collect Result_Fill1',
       'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1',
       'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1',
       'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1',
       'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1',
       'HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1',
       'HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1',
       'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1',
       'Head Purge Position Z Collect Result_Fill1',
       'PalletID Collect Result_Fill1',
       'CURE END POSITION X Collect Result_Fill2',
       'CURE END POSITION Z Collect Result_Fill2',
       'CURE SPEED Collect Result_Fill2',
       'CURE STANDBY POSITION Z Collect Result_Fill2',
       'CURE START POSITION X Collect Result_Fill2',
       'CURE START POSITION Z Collect Result_Fill2',
       'HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2',
       'HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2',
       'HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2',
       'HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2',
       'HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2',
       'Head Purge Position Z Collect Result_Fill2',
       'Machine Tact time Collect Result_Fill2',
       'PalletID Collect Result_Fill2', 'MEAN Dispense Volume_Dam',
       'SUM Dispense Volume_Dam', 'DIV Dispense Volume(Stage2)_Dam',
       'DIV Dispense Volume(Stage3)_Dam', 'SUM DISCHARGED TIME OF RESIN_Fill1',
       'MEAN Dispense Volume_Fill1', 'STD Dispense Volume_Fill1',
       'DIV1 THICKNESS_Dam', 'DIV2 THICKNESS_Dam', 'SUM Pressure_AutoClave',
       'MEAN Pressure_AutoClave', 'DIV1 Pressure_AutoClave',
       'DIV2 Pressure_AutoClave'

       ]

# 개수 확인 (85개)
print(len(cols))

In [ ]:
# target 컬럼 라벨링 처리

df['target']=np.where(df['target']=='AbNormal',1,0)
test_df['target']=np.where(test_df['target']=='AbNormal',1,0)

In [ ]:
# 데이터 분할 및 복제

X=df[cols]
y=df[['target']]
test_x=test_df.drop(columns=['Set ID','target'])

input_X = X.copy()
input_y = y.copy()
input_X_test = test_x.copy()
input_X_test=input_X_test[cols]

In [ ]:
# 제출 데이터 불러오기

sub = pd.read_csv("submission.csv")

### 모델 학습

선정 모델 : LightGBM

이유

1. Gradient Boosting 알고리즘 기반으로 복잡한 데이터셋에서도 효과적으로 잘 작동하고 높은 예측 정확도를 보임

2. 데이터와 피처를 매우 빠르게 처리

3. 범주형 데이터 임의 처리

4. 하이퍼파라미터 조정이 유연하여 모델 성능을 최적화가 가능하여 성능을 더 세밀하게 조정 가능

5. 빠른 학습 시간을 자랑

In [ ]:
# 모델 학습 및 예측

import lightgbm as lgb

input_X = X.copy()
input_y = y.copy()
input_X_test = test_x.copy()
input_X_test=input_X_test[cols]

# 범주형 피처 인덱스 리스트 작성
cat_features = input_X.select_dtypes(include=['object']).columns.tolist()

# object 타입을 category 타입으로 변환
for col in cat_features:
    input_X[col] = input_X[col].astype('category')
    input_X_test[col] = input_X_test[col].astype('category')

model = lgb.LGBMClassifier( learning_rate=0.019048936944048345, random_state=42, verbose=-1,
                               n_estimators = 481, num_leaves = 100, max_depth = 13, min_child_samples = 74, subsample = 0.9422125802598914, colsample_bytree = 0.53092502712293,
                                reg_alpha = 2.475051708653131, reg_lambda = 2.5852112412936266, scale_pos_weight = 6.78337274078157
                               )

model.fit(input_X, input_y, categorical_feature=cat_features)

y_pred = model.predict(input_X_test)

# predictions를 sub 데이터프레임의 'target' 컬럼에 저장
sub['target'] = y_pred

# 결과 확인
sub['target'].replace({0:'Normal',1:'AbNormal'},inplace=True)
sub['target'].value_counts()

In [ ]:
# 추가 핸드라벨링

Check = pd.read_csv(os.path.join(ROOT_DIR, "new_test.csv"))

Check['Equipment_Dam']=Check['Equipment_Dam'].apply(lambda x: x[-1])
Check['Equipment_Fill1']=Check['Equipment_Fill1'].apply(lambda x: x[-1])
Check['Equipment_Fill2']=Check['Equipment_Fill2'].apply(lambda x: x[-1])

Check.loc[((Check['Receip No Collect Result_Dam']!=Check['Receip No Collect Result_Fill1'])|(Check['Receip No Collect Result_Dam']!=Check['Receip No Collect Result_Fill2'])|(Check['Receip No Collect Result_Fill1']!=Check['Receip No Collect Result_Fill2'])|(Check['Production Qty Collect Result_Dam']!=Check['Production Qty Collect Result_Fill1'])|(Check['Production Qty Collect Result_Dam']!=Check['Production Qty Collect Result_Fill2'])|(Check['Production Qty Collect Result_Fill1']!=Check['Production Qty Collect Result_Fill2'])|(Check['PalletID Collect Result_Dam']!=Check['PalletID Collect Result_Fill1'])|(Check['PalletID Collect Result_Dam']!=Check['PalletID Collect Result_Fill2'])|(Check['PalletID Collect Result_Fill1']!=Check['PalletID Collect Result_Fill2'])|(Check['Equipment_Dam']!=Check['Equipment_Fill1'])|(Check['Equipment_Dam']!=Check['Equipment_Fill2'])|(Check['Equipment_Fill1']!=Check['Equipment_Fill2']))].head(5)

#몇개 바꿀 수 있는지 확인
sub.loc[
   Check.loc[((Check['Receip No Collect Result_Dam']!=Check['Receip No Collect Result_Fill1'])|(Check['Receip No Collect Result_Dam']!=Check['Receip No Collect Result_Fill2'])|(Check['Receip No Collect Result_Fill1']!=Check['Receip No Collect Result_Fill2'])|(Check['Production Qty Collect Result_Dam']!=Check['Production Qty Collect Result_Fill1'])|(Check['Production Qty Collect Result_Dam']!=Check['Production Qty Collect Result_Fill2'])|(Check['Production Qty Collect Result_Fill1']!=Check['Production Qty Collect Result_Fill2'])|(Check['PalletID Collect Result_Dam']!=Check['PalletID Collect Result_Fill1'])|(Check['PalletID Collect Result_Dam']!=Check['PalletID Collect Result_Fill2'])|(Check['PalletID Collect Result_Fill1']!=Check['PalletID Collect Result_Fill2'])|(Check['Equipment_Dam']!=Check['Equipment_Fill1'])|(Check['Equipment_Dam']!=Check['Equipment_Fill2'])|(Check['Equipment_Fill1']!=Check['Equipment_Fill2']))].index,'target'].value_counts()

#수정
sub.loc[
   Check.loc[((Check['Receip No Collect Result_Dam']!=Check['Receip No Collect Result_Fill1'])|(Check['Receip No Collect Result_Dam']!=Check['Receip No Collect Result_Fill2'])|(Check['Receip No Collect Result_Fill1']!=Check['Receip No Collect Result_Fill2'])|(Check['Production Qty Collect Result_Dam']!=Check['Production Qty Collect Result_Fill1'])|(Check['Production Qty Collect Result_Dam']!=Check['Production Qty Collect Result_Fill2'])|(Check['Production Qty Collect Result_Fill1']!=Check['Production Qty Collect Result_Fill2'])|(Check['PalletID Collect Result_Dam']!=Check['PalletID Collect Result_Fill1'])|(Check['PalletID Collect Result_Dam']!=Check['PalletID Collect Result_Fill2'])|(Check['PalletID Collect Result_Fill1']!=Check['PalletID Collect Result_Fill2'])|(Check['Equipment_Dam']!=Check['Equipment_Fill1'])|(Check['Equipment_Dam']!=Check['Equipment_Fill2'])|(Check['Equipment_Fill1']!=Check['Equipment_Fill2']))].index,'target']='AbNormal'

sub['target'].value_counts()

In [ ]:
# 제출 파일 저장
sub.to_csv("submission.csv", index=False)